# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

In [2]:
# Import dependencies
%pip install pymongo
from pymongo import MongoClient
from pymongo import UpdateOne
from pprint import pprint

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.0/732.0 kB 20.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['artifacts', 'establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'BusinessTypeID': 1,
 'RatingValue': None,
 '_id': ObjectId('6740081a2ecd7f382f7aba33'),
 'cuisine': 'Halal',
 'geocode': {'latitude': None, 'longitude': None},
 'location': {'address': '123 Greenwich Ave',
              'city': 'Greenwich',
              'state': 'CT',
              'zipcode': '06830'},
 'name': 'Penang Flavours',
 'opened': True,
 'ratings': []}


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "cuisine": "Halal",
    "location": {
        "address": "123 Greenwich Ave",
        "city": "Greenwich",
        "state": "CT",
        "zipcode": "06830"
    },
    "ratings": [],  # No ratings yet
    "opened": True
}

In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)


In [11]:
# Check that the new restaurant was inserted
count = establishments.count_documents({"name": "Penang Flavours"})

if count > 0:
    print(f"{count} restaurant(s) named 'Penang Flavours' found in the database.")
else:
    print("No restaurant named 'Penang Flavours' found.")


27 restaurant(s) named 'Penang Flavours' found in the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
result = db.artifacts.find_one(query, fields)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
# Debugging: Check if result contains BusinessTypeID
print("Result from the query:", result)

# Update the new restaurant with the correct BusinessTypeID
if result and 'BusinessTypeID' in result:
    update_result = db.establishments.update_one(
        {'name': 'Penang Flavours'},
        {'$set': {'BusinessTypeID': result['BusinessTypeID']}}
    )
    # Display update results
    print(f"Matched {update_result.matched_count} document(s).")
    print(f"Modified {update_result.modified_count} document(s).")
else:
    print("BusinessTypeID not found in the query result. Update aborted.")

# Confirm that the new restaurant was updated
updated_doc = db.establishments.find_one(
    {'name': 'Penang Flavours'}, 
    {'name': 1, 'BusinessTypeID': 1, '_id': 0}
)
print("Updated Document:", updated_doc)


Result from the query: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
Matched 1 document(s).
Modified 0 document(s).
Updated Document: {'name': 'Penang Flavours', 'BusinessTypeID': 1}


In [15]:
# Confirm that the new restaurant was updated
pprint(db.establishments.find_one({'name': 'Penang Flavours'}, {'name': 1, 'BusinessTypeID': 1, '_id': 0}))


{'BusinessTypeID': 1, 'name': 'Penang Flavours'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f"There are {dover_count} documents with LocalAuthorityName as 'Dover'.")

There are 0 documents with LocalAuthorityName as 'Dover'.


In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [19]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f"There are {dover_count} documents with LocalAuthorityName as 'Dover' after deletion.")

There are 0 documents with LocalAuthorityName as 'Dover' after deletion.


In [20]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'BusinessTypeID': 1,
 'RatingValue': None,
 '_id': ObjectId('6740081a2ecd7f382f7aba33'),
 'cuisine': 'Halal',
 'geocode': {'latitude': None, 'longitude': None},
 'location': {'address': '123 Greenwich Ave',
              'city': 'Greenwich',
              'state': 'CT',
              'zipcode': '06830'},
 'name': 'Penang Flavours',
 'opened': True,
 'ratings': []}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [436]:
# Change the data type from String to Decimal for longitude and latitude

operations = []
for doc in establishments.find({'geocode.latitude': {'$type': 'string'}}):
    try:
        latitude = float(doc['geocode']['latitude'])
        longitude = float(doc['geocode']['longitude'])
        operations.append(
            UpdateOne(
                {'_id': doc['_id']},
                {'$set': {'geocode.latitude': latitude, 'geocode.longitude': longitude}}
            )
        )
    except (ValueError, TypeError):
        pass
if operations:
    establishments.bulk_write(operations)

Use `update_many` to convert `RatingValue` to integer numbers.

In [437]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [438]:
# Change the data type from String to Integer for RatingValue

operations = []
for doc in establishments.find({'RatingValue': {'$type': 'string', '$ne': None}}):
    try:
        rating = int(doc['RatingValue'])
        operations.append(
            UpdateOne(
                {'_id': doc['_id']},
                {'$set': {'RatingValue': rating}}
            )
        )
    except ValueError:
        pass
if operations:
    establishments.bulk_write(operations)

In [439]:
# Check that the coordinates and rating value are now numbers
pprint(db.establishments.find_one({}, {'geocode.latitude': 1, 'geocode.longitude': 1, 'RatingValue': 1, '_id': 0}))

{'RatingValue': None, 'geocode': {'latitude': None, 'longitude': None}}
